# Análisis de vuelos
# Scraping y extracción

 En este laboratorio vamos a abordar la extracción de información mediante web scraping, para luego extraer esa información y almacenar en archivo csv.


## Instalación e Importación de librerías

In [ ]:
!pip install xlsxwriter
!pip install tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 8.5 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from datetime import datetime
import os

## Declaración de constantes

In [ ]:
# url = "https://failbondi.fail/?date="

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

year = "2025"

months_max_days = { "01": 31, "02": 28, "03": 31, "04": 30, "05": 31, "06": 30, "07": 31, "08": 31, "09": 30, "10": 31, "11": 30, "12": 31 }
month_days = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11","12","13","14","15","16","17","18","19","20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"]
FLYBONDI = "FO"
JETSMART = "WJ"

In [ ]:
def generate_url(date, company):
    return f"https://failbondi.fail/?date={date}&aerolinea={company}"


## Funciones reutilizables

In [ ]:
def get_html_from_url(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        return soup

    return None

def scraping_vuelos(html, date, company):
    """
    Extracts flight data from an HTML table and structures it into a DataFrame.

    Args:
        html (bs4.BeautifulSoup): The BeautifulSoup object containing the parsed HTML.
        date (str or datetime): The reference date to be associated with the scraped data.

    Returns:
        pd.DataFrame: A cleaned DataFrame containing flight records, including
                      standardized dates and month extraction for grouping.
    """
    # 1. Extract headers
    headers = [th.text.strip() for th in html.find('thead').find_all('th')]

    # 2. Extraer filas
    rows = []
    table_body = html.find('tbody')
    for tr in table_body.find_all('tr'):
        cells = [td.text.strip() for td in tr.find_all('td')]
        rows.append(cells)

    # 3. Crear DataFrame
    df = pd.DataFrame(rows, columns=headers)
    df['fecha'] = date
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['mes'] = df['fecha'].dt.month
    df['empresa'] = company

    return df


def get_report_by_month(year_month, max_days, company):
    lista_dfs = []
    for i in range(max_days):
        date = f"{year_month}-{month_days[i]}"
        url_link = generate_url(date, company=company)

        time.sleep(random.uniform(1.5, 3.5))

        main_content = get_html_from_url(url_link, headers)

        try:
            df_iteracion = scraping_vuelos(main_content, date, company)
            if not df_iteracion.empty:
                lista_dfs.append(df_iteracion)
        except Exception as e:
            print(f"Error en fecha {date}: {e}")
            time.sleep(10)

    if not lista_dfs: return pd.DataFrame()

    df_month = pd.concat(lista_dfs, ignore_index=True)
    print(f"[{year_month}] - Filas obtenidas: {len(df_month)}")
    return df_month


In [ ]:
lista_dfs = []

inicio_peticion_total = time.time()
company = JETSMART

for month, max_days in months_max_days.items():
    year_month = f"{year}-{month}"
    print(f"Iniciando extracción de: {year_month}")

    inicio_peticion = time.time()

    # lista_dfs.append(get_report_by_month(year_month, max_days))
    df_mensual = get_report_by_month(year_month, max_days, company=company)

    fin_peticion = time.time()
    duracion = fin_peticion - inicio_peticion
    hora_actual = datetime.now().strftime('%H:%M:%S')

    filename = f"raw_vuelos_{company}_{year_month}.csv"
    df_mensual.to_csv(filename, index=False)
    print(f"Archivo {filename} guardado con éxito.")

    # Descanso largo entre meses para "enfriar" la IP
    time.sleep(random.uniform(10, 20))

    print(f"[{hora_actual}] Finalizado: {year_month} | Tiempo: {duracion:.2f}s")

fin_peticion_total = time.time()
duracion_total = fin_peticion_total - inicio_peticion_total

print("Duración total del proceso: ", duracion_total)


Iniciando extracción de: 2025-01
[2025-01] - Filas obtenidas: 1696
Archivo raw_vuelos_WJ_2025-01.csv guardado con éxito.
[14:17:17] Finalizado: 2025-01 | Tiempo: 87.89s
Iniciando extracción de: 2025-02
[2025-02] - Filas obtenidas: 1623
Archivo raw_vuelos_WJ_2025-02.csv guardado con éxito.
[14:18:55] Finalizado: 2025-02 | Tiempo: 80.56s
Iniciando extracción de: 2025-03
[2025-03] - Filas obtenidas: 1739
Archivo raw_vuelos_WJ_2025-03.csv guardado con éxito.
[14:20:32] Finalizado: 2025-03 | Tiempo: 86.50s
Iniciando extracción de: 2025-04
[2025-04] - Filas obtenidas: 1832
Archivo raw_vuelos_WJ_2025-04.csv guardado con éxito.
[14:22:18] Finalizado: 2025-04 | Tiempo: 87.89s
Iniciando extracción de: 2025-05
[2025-05] - Filas obtenidas: 1844
Archivo raw_vuelos_WJ_2025-05.csv guardado con éxito.
[14:23:57] Finalizado: 2025-05 | Tiempo: 86.73s
Iniciando extracción de: 2025-06
[2025-06] - Filas obtenidas: 1764
Archivo raw_vuelos_WJ_2025-06.csv guardado con éxito.
[14:25:37] Finalizado: 2025-06 | T

In [ ]:
import glob
import os


def export_and_unify_files(folder_path, company, format="parquet", prefijo=None):
    # 1. Buscar todos los archivos mensuales'
    patron = "*.csv"

    if prefijo:
        patron = f"{prefijo}*.csv"

    archivos = glob.glob(os.path.join(folder_path, patron))
    archivos.sort() # Para mantener el orden cronológico

    lista_dfs = []
    for archivo in archivos:
        print(f"Leyendo: {archivo}")
        df = pd.read_csv(archivo)
        lista_dfs.append(df)

    # 2. Unir todos (pandas ignora los headers repetidos y crea un solo esquema)
    df_final = pd.concat(lista_dfs, ignore_index=True)

    nombre_archivo = f"vuelos_anual_{company}_consolidado_{YEAR}"
    # 3. Exportar según formato
    if format == "csv":
        df_final.to_csv(nombre_archivo + ".csv", index=False)
    elif format == "parquet":
        # Requiere: pip install pyarrow fastparquet
        df_final.to_parquet(nombre_archivo + ".parquet", index=False)
    elif format == "excel":
        df_final.to_excel(nombre_archivo + ".xlsx", index=False)

    print(f"Consolidación exitosa. Filas totales: {len(df_final)}")
    return df_final


In [ ]:
%pwd

'/content'

In [ ]:
export_and_unify_files('/content', company)
export_and_unify_files('/content', company, 'csv')


Leyendo: /content/raw_vuelos_WJ_2025-01.csv
Leyendo: /content/raw_vuelos_WJ_2025-02.csv
Leyendo: /content/raw_vuelos_WJ_2025-03.csv
Leyendo: /content/raw_vuelos_WJ_2025-04.csv
Leyendo: /content/raw_vuelos_WJ_2025-05.csv
Leyendo: /content/raw_vuelos_WJ_2025-06.csv
Leyendo: /content/raw_vuelos_WJ_2025-07.csv
Leyendo: /content/raw_vuelos_WJ_2025-08.csv
Leyendo: /content/raw_vuelos_WJ_2025-09.csv
Leyendo: /content/raw_vuelos_WJ_2025-10.csv
Leyendo: /content/raw_vuelos_WJ_2025-11.csv
Leyendo: /content/raw_vuelos_WJ_2025-12.csv
Consolidación exitosa. Filas totales: 23645
Leyendo: /content/raw_vuelos_WJ_2025-01.csv
Leyendo: /content/raw_vuelos_WJ_2025-02.csv
Leyendo: /content/raw_vuelos_WJ_2025-03.csv
Leyendo: /content/raw_vuelos_WJ_2025-04.csv
Leyendo: /content/raw_vuelos_WJ_2025-05.csv
Leyendo: /content/raw_vuelos_WJ_2025-06.csv
Leyendo: /content/raw_vuelos_WJ_2025-07.csv
Leyendo: /content/raw_vuelos_WJ_2025-08.csv
Leyendo: /content/raw_vuelos_WJ_2025-09.csv
Leyendo: /content/raw_vuelos_WJ_

,Vuelo,Ruta,Hora Programada,Hora Real,Demora en despegar,fecha,mes,empresa
0,WJ 3169,Aeroparque → Neuquen,18:29,00:55 +1,6hs 26min tarde,2025-01-01,1,WJ
1,WJ 3142,Aeroparque → Iguazú,19:20,22:25,3hs 5min tarde,2025-01-01,1,WJ
2,WJ 3145,Iguazú → Ezeiza,21:41,00:38 +1,2hs 57min tarde,2025-01-01,1,WJ
3,WJ 3165,Aeroparque → Neuquen,16:40,18:46,2hs 6min tarde,2025-01-01,1,WJ
4,WJ 3820,Aeroparque → Florianopolis,16:55,18:56,2hs 1min tarde,2025-01-01,1,WJ
...,...,...,...,...,...,...,...,...
23640,WJ 3011,Salta → Aeroparque,21:29,21:24,adelantado 5min,2025-12-31,12,WJ
23641,WJ 3145,Iguazú → Ezeiza,13:57,13:50,adelantado 7min,2025-12-31,12,WJ
23642,WJ 3056,Bariloche → Ezeiza,22:52,22:42,adelantado 10min,2025-12-31,12,WJ
23643,WJ 3155,Iguazú → Ezeiza,22:34,22:16,adelantado 18min,2025-12-31,12,WJ


Este notebook finaliza con el archivo resultante de todos los vuelos que hay en la página de todo el año 2025.

Consideraciones al realizar web scraping, para evitar que la IP sea baneada se considero agregar tiempos de espera ```sleep```, durante las peticiones por fechas y en caso que surja algún error se agrega tiempo extra.

